In [1]:
from sympy import sqrt, I, series, limit, symbols, factorial
from sympy.core.add import Add
from sympy.core.mul import Mul
from sympy.physics.secondquant import B, Bd, CreateBoson, AnnihilateBoson, Commutator, KroneckerDelta

In [2]:
# Eliminates all powers
def flatten(expr):
    expr = expr.expand()
    if expr.is_Add: return Add(*[flatten(arg) for arg in expr.args])
    elif expr.is_Mul: 
        factors = []
        for arg in expr.args:
            if arg.is_Pow and arg.exp.is_Integer and arg.exp > 0:
                factors.extend([arg.base] * int(arg.exp))
            else:
                factors.append(flatten(arg))
        return Mul(*factors, evaluate=False)
    elif expr.is_Pow and expr.exp.is_Integer and expr.exp > 0:
        return Mul(*[flatten(expr.base)] * int(expr.exp), evaluate=False)
    return expr

# Puts a polynomial expression of bosonic operators into the equivalent normal ordered form
def wick(expr):
    expr = flatten(expr)
    if expr.is_Add: return Add(*[wick(arg) for arg in expr.args])
    elif expr.is_Mul:
        commuting = [x for x in expr.args if not isinstance(x, AnnihilateBoson) and not isinstance(x, CreateBoson)]
        noncommuting = [x for x in expr.args if isinstance(x, AnnihilateBoson) or isinstance(x, CreateBoson)]
        n = len(noncommuting)
        if n == 0 or n == 1: return expr
        else:
            for i in range(n-1):
                c = Commutator(noncommuting[i], noncommuting[i+1])
                if isinstance(c, KroneckerDelta): c = 0
                if isinstance(noncommuting[i], AnnihilateBoson) and isinstance(noncommuting[i+1], CreateBoson):
                    expr1 = [c] + noncommuting[:i] + noncommuting[i+2:]
                    expr2 = noncommuting[:i] + [noncommuting[i+1], noncommuting[i]] + noncommuting[i+2:]
                    return wick(Mul(*commuting) * (wick(Mul(*expr1)) + wick(Mul(*expr2))))
            return expr
    else:
        return expr

# Computes the vacuum expectation value of a polynomial expression of bosonic operators
vev = lambda expr: Add(*[x for x in wick(expr).args if not any(isinstance(y, AnnihilateBoson) or isinstance(y, CreateBoson) for y in x.args)])

In [6]:
# Verifies the above functions work by using them to reproduce the main results of the paper:
# Quantum Theory of the Josephson Junction between Finite Islands

ng, x = symbols('n_g x', real = True)
N, EJ, EC = symbols("N E_J E_C", positive = True)
u_s, u_p, u_m, u_0 = symbols('u_s, u_p, u_m, u_0')
eps = sqrt(2*EC*EJ + EJ**2/N**2)
u_p = (EJ + N * eps) / sqrt(4 * N * eps * EJ)
u_m = (EJ - N * eps) / sqrt(4 * N * eps * EJ)
u_0 = ng * sqrt(2 * EC**2 * EJ / eps**3)
u_s = u_p + u_m

b = B('b')
bd = Bd('b')
a = u_p*b - u_m*bd + I*u_0*(u_p+u_m)
ad = u_p*bd - u_m*b - I*u_0*(u_p+u_m)
p = (a-ad)/(I*sqrt(2))
pp = p - ng/sqrt(N)

expr = -EC/2*(b*pp*ad*p*a*bd - pp*ad*p*a)
freq = eps + vev(expr)
assert((freq - series(freq, ng, 0, 3)).simplify()==0)
assert(freq.diff(ng).subs(ng,0)==0)
freq_0 = limit(freq.subs(ng, 0).subs(N, x * EJ/EC), x, "oo")
freq_2 = limit((N**2 * (freq.diff(ng, 2)/2).subs(ng, 0)).subs(N, x * EJ/EC), x, "oo")*(ng/N)**2
freq = (limit(((freq_0+freq_2)/sqrt(EC * EJ)).subs(EC, x * EJ), x, 0) * sqrt(EC * EJ)).simplify()
print('Transmon qubit frequency times Planck\'s constant:')
display(freq)

zeroth_order = vev(sqrt(N)*p)
first_order = vev(-ad*p*a/(4*sqrt(N)))
for k in range(1,5): first_order += EC*sqrt(N)/(2*eps*k*factorial(k)) * vev(p*bd**k) * (vev(b**k*pp*ad*p*a) + vev(b**k*ad*p*a*pp))
all_orders = (zeroth_order + first_order).simplify()
all_orders.diff(ng, 3).subs(ng, 0).simplify()
suscept = all_orders.diff(ng)
assert((suscept - series(suscept, ng, 0, 3)).simplify() == 0)
assert(suscept.diff(ng).subs(ng, 0) == 0)
suscept_0 = limit(suscept.subs(ng, 0).subs(N, x * EJ/EC), x, "oo")
suscept_2 = limit((N**4 * (suscept.diff(ng, 2)/2).subs(ng, 0)).subs(N, x * EJ/EC), x, "oo")*(ng**2)/(N**4)
suscept = suscept_0 + suscept_2
print('Transmon dimensionless charge susceptibility:')
display(suscept.simplify())

Transmon qubit frequency times Planck's constant:


sqrt(2)*sqrt(E_C)*sqrt(E_J)*(N**2 - n_g**2/4)/N**2

Transmon dimensionless charge susceptibility:


1 - 3*E_J*n_g**2/(4*E_C*N**4)